# FOOD VECTOR SEARCH

In [ ]:
!pip install PyPDF2
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.9 MB/s eta 0

Lire un pdf et appliquer du embedding

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6z4EmYqfgIxyYLGukHedvA/foodMenu.pdf

--2025-02-08 22:28:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6z4EmYqfgIxyYLGukHedvA/foodMenu.pdf
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212668 (208K) [application/pdf]
Saving to: ‘foodMenu.pdf.1’

foodMenu.pdf.1      100%[===================>] 207.68K  --.-KB/s    in 0.1s    

2025-02-08 22:28:25 (1.71 MB/s) - ‘foodMenu.pdf.1’ saved [212668/212668]



In [ ]:
from PyPDF2 import PdfReader

# Ouvrir le fichier PDF
reader = PdfReader("foodMenu.pdf")

# Obtenir le nombre de pages
number_of_pages = len(reader.pages)

# Extraire le texte de toutes les pages
text = ""
for page_num in range(number_of_pages):
    page = reader.pages[page_num]
    text += page.extract_text()

# Afficher le texte
text = text.replace("\n", " ").replace("  ", " ")

In [ ]:
text_embedding = client.feature_extraction(text)

In [ ]:
text_embedding

array([-3.90361249e-02, -2.07622480e-02,  6.97398111e-02,  3.88702080e-02,
        1.15701379e-02,  3.96985039e-02, -1.14835743e-02,  2.36800611e-02,
        7.52899274e-02, -1.01693475e-03, -5.63242249e-02, -4.81835902e-02,
       -4.90042455e-02, -6.16592318e-02, -8.40486307e-03, -9.93115548e-03,
        1.18030079e-01, -2.64369175e-02,  1.50149902e-02, -4.57093827e-02,
        5.79187721e-02, -2.06559096e-02,  4.79550175e-02,  5.78375347e-02,
        4.76792343e-02,  5.41423187e-02,  8.06326494e-02, -7.20125362e-02,
       -8.95031840e-02, -2.16865297e-02, -3.84214371e-02, -2.64533609e-02,
       -6.23111892e-03, -5.38338870e-02,  1.35312872e-02, -7.81862363e-02,
        1.19988374e-01, -5.57969557e-03, -2.73074722e-03, -2.87858974e-02,
        2.26520002e-03,  8.21501315e-02, -2.68454012e-02, -3.65535207e-02,
        1.84803281e-03, -9.08461493e-03, -4.62096557e-02, -3.98545107e-03,
        4.14622910e-02,  2.26672459e-02, -2.88198553e-02,  2.57832650e-03,
        2.14889590e-02, -

# Recommandation based on user query

### Importation des modules

In [ ]:
# Implémentation de la fonction de récupérer des clés d'api : huggingface_api et gemini_api
from google.colab import userdata
huggingface_api = userdata.get('huggingface')
gemini_api = userdata.get('GOOGLE_API_KEY')

In [ ]:
import chromadb
import huggingface_hub
from huggingface_hub import InferenceClient
import PyPDF2
import os
import re
import json

In [ ]:
chroma_client = chromadb.Client()
hf_client_emb = InferenceClient(
    "sentence-transformers/all-MiniLM-L6-v2",
    token=huggingface_api,
)

### Importation des données

### Début du code

In [ ]:
from foodDataSet import foodItems

In [ ]:
collectionName = "food_collection"

In [ ]:
# fonction de génération des embeddings
def generateEmbeddings(texts):
  texts_embeddings = hf_client_emb.feature_extraction(texts)
  return texts_embeddings

### Développer la fonction du critère de recherche

#### Avec Gemini

In [ ]:
import google.generativeai as genai

genai.configure(api_key=gemini_api)
model = genai.GenerativeModel('gemini-pro')


In [ ]:
# recherche de critère avec l'api de gemini
def extractFilterCriteria_gemini(query):

  labels = ["vegan", "non-vegan", "vegetarian", "non-vegetarian", "pescatarian", "omnivore", "paleo", "ketogenic", "chinese", "indian", "japanese", "autre"]


  prompt = f"""
    You are a powerful AI trained to classify text into one of the following categories.
    Here are the categories: {labels}.

    Your task is to determine which category best fits the following text.

    Text: "{query}"

    Please respond with one of the categories listed above that best describes the text.

    """

  response = model.generate_content(prompt)

  return response.text


extractFilterCriteria_gemini("I want to eat Chocolate")

'non-vegan'

#### Avec Facebook Bart (open-source)

In [ ]:
from transformers import pipeline

# Créer un pipeline pour la classification de texte
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Fonction pour classer le texte
def classifyText(text, labels):
    result = classifier(text, candidate_labels=labels)
    return result

# fonction de filtrage de critère
def extractFilterCriteria(query):

  criteria = {
      "diet":[], "cuisine":[]
  }
  dietlabels = ["vegan", "non-vegan", "vegetarian", "non-vegetarian", "pescatarian", "omnivore", "paleo", "ketogenic"]
  cuisineLabels = ["chinese", "indian", "japanese"]

  dietResult = classifyText(query, dietlabels)
  print(dietResult)
  highestDietScoreLabel = dietResult["labels"][0]
  dietScore = dietResult['scores'][0]

  if dietScore > 0.8:
    criteria["diet"].append(highestDietScoreLabel)
  else:
    cuisineResult = classifyText(query, cuisineLabels)
    print(cuisineResult)
    highestCuisineScoreLabel = cuisineResult["labels"][0]
    cuisineScore = cuisineResult['scores'][0]

    if cuisineScore > 0.8:
      criteria["cuisine"].append(highestCuisineScoreLabel)

  print('Extracted Filter Criteria:', criteria)

  return criteria

extractFilterCriteria("j'aime les repas de l'asie de l'Est")

{'sequence': "j'aime les repas de l'asie de l'Est", 'labels': ['non-vegetarian', 'non-vegan', 'pescatarian', 'paleo', 'omnivore', 'ketogenic', 'vegetarian', 'vegan'], 'scores': [0.3223329484462738, 0.27310770750045776, 0.22097794711589813, 0.06264303624629974, 0.057116150856018066, 0.033554162830114365, 0.021428048610687256, 0.008839981630444527]}
{'sequence': "j'aime les repas de l'asie de l'Est", 'labels': ['indian', 'japanese', 'chinese'], 'scores': [0.5963707566261292, 0.22059422731399536, 0.1830349862575531]}
Extracted Filter Criteria: {'diet': [], 'cuisine': []}


{'diet': [], 'cuisine': []}

### Recherche de similarité (interrogation)

In [ ]:
def performSimilaritySearch(collection, queryTerm):

  try:

    query_embedding = generateEmbeddings([queryTerm])

    results = collection.query(
        query_embeddings= query_embedding,
        n_results= 5,
    )

    if len(results) == 0:
      return "Auun document trouvé"
    else:

      top_foodItems = [
          {
              'ids': [food_id for food_id in results['ids'][0]],
              'distances': [distance for distance in results['distances'][0]],
              'food_names': [food_name['food_name'] for food_name in results['metadatas'][0]]
          }
          for index, id in enumerate(results['ids'][0])
      ]

    return top_foodItems

  except Exception as e:
    print("Exception:", e)


### Fonction principale

In [ ]:
def main(collection_name, query):

  try:

    collection = chroma_client.get_or_create_collection(name=collection_name)
    print("Collection created or retrieved successfully!")

    uniqueIds = [f"{food['food_id']}_{i}" for i, food in enumerate(foodItems)]
    foodTexts = [
      f"{food['food_name']}. {food['food_description']}. Ingredients: {', '.join(food['food_ingredients'])}"
      for food in foodItems
    ]
    embeddingsData = generateEmbeddings(foodTexts)
    metadata = [
      {
        'food_name': food['food_name']
      } for food in foodItems
    ]

    # ajouter le ids, documents et embeddings à la collection
    collection.add(
        ids = uniqueIds,
        documents = foodTexts,
        metadatas = metadata,
        embeddings= embeddingsData
    )

    #filterCriteria = extractFilterCriteria_gemini(query)
    #print(filterCriteria)
    initialResults = performSimilaritySearch(collection, query)


    for index, item in enumerate(initialResults[0]["food_names"]):
      print(f"Top {index + 1} Recommended Food Name: {item}")

  except Exception as e:

    print("Exception:", e)

In [ ]:
chroma_client.delete_collection(collectionName)
query = "egg, butter and rice"
main(collectionName, query)

Collection created or retrieved successfully!
Top 1 Recommended Food Name: Salted Roasted Rice Crackers
Top 2 Recommended Food Name: Garlic Bread
Top 3 Recommended Food Name: Salted Roasted Mixed Seeds
Top 4 Recommended Food Name: Pad Thai
Top 5 Recommended Food Name: Vegetable Stir Fry


# Food recommendations from a PDF recipe

### Uploader les données

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/HoMe0o66TlJJ-WrIcR_8HQ/Chocolate-torte-Recipe.pdf

--2025-02-10 08:22:25--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/HoMe0o66TlJJ-WrIcR_8HQ/Chocolate-torte-Recipe.pdf
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197687 (193K) [application/pdf]
Saving to: ‘Chocolate-torte-Recipe.pdf’

Chocolate-torte-Rec 100%[===================>] 193.05K  --.-KB/s    in 0.1s    

2025-02-10 08:22:25 (1.67 MB/s) - ‘Chocolate-torte-Recipe.pdf’ saved [197687/197687]



In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/GvUxpXUD-oy1h5z-qKoVFg/crumble-pie.pdf

--2025-02-10 08:22:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/GvUxpXUD-oy1h5z-qKoVFg/crumble-pie.pdf
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248041 (242K) [application/pdf]
Saving to: ‘crumble-pie.pdf’

crumble-pie.pdf     100%[===================>] 242.23K  --.-KB/s    in 0.1s    

2025-02-10 08:22:29 (1.70 MB/s) - ‘crumble-pie.pdf’ saved [248041/248041]



In [ ]:
collectionName = "recipe_food"

In [ ]:
from PyPDF2 import PdfReader

def extractTextFromPDF(filepath):

  # Ouvrir le fichier PDF
  reader = PdfReader(filepath)

  # Obtenir le nombre de pages
  number_of_pages = len(reader.pages)

  # Extraire le texte de toutes les pages
  text = ""
  for page_num in range(number_of_pages):
      page = reader.pages[page_num]
      text += page.extract_text()

  # Afficher le texte
  text = text.replace("\n", " ").replace("  ", " ")

  return text
#text = extractTextFromPDF("crumble-pie.pdf")

In [ ]:
def promptUserInput(query):
  path = str(input("Entrer le chemin du fichier : "))
  return path

In [ ]:
import ast

def extractIngredients_gemini(text):


  prompt = f"""

    Act as an expert in text processing and structured data extraction. Your task is to extract
    only the ingredients from the following recipe text while maintaining without their quantities
    and units.  Remove any quantities !!!,  Return the list in a structured format

    Here is the text:{text}

    Return the result as a python list where each ingredient is a separate item !
    The result must be only a list like : ['first ingredient', 'second ingredient', ...] !!!

    """

  response = model.generate_content(prompt)

  liste_ingredients = ast.literal_eval(response.text)

  ingredients = list(set(liste_ingredients))

  return ingredients

#extractFilterCriteria_gemini(text)

In [ ]:
def storeEmbeddingsInChromaDB(foodItems):

  foodEmbeddings = []
  metadatas = [
      {
        'food_name': food['food_name']
      } for food in foodItems
    ]

  for item in foodItems:
    texte = " ".join(item["food_ingredients"])

    embedding = generateEmbeddings(texte)

    foodEmbeddings.append(embedding)

  ids = [f"{food['food_id']}_{i}" for i, food in enumerate(foodItems)]

  foodTexts = [
      f"Ingredients: {', '.join(food['food_ingredients'])}"
      for food in foodItems
    ]

  try:

    collection = chroma_client.get_or_create_collection(name=collectionName)
    print("ok")

    collection.add(
        ids = ids,
        documents = foodTexts,
        metadatas = metadatas,
        embeddings= foodEmbeddings
    )
    print("Embeddings stocker dans chromaDB ")

  except Exception as e:
    print("Exception store :", e)

  return collection

In [ ]:
def main():

  try:

    collection = storeEmbeddingsInChromaDB(foodItems)
    print(collection)
    #filepath = promptUserInput(query)
    text = extractTextFromPDF('Chocolate-torte-Recipe.pdf')
    ingredients = extractIngredients_gemini(text)

    if len(ingredients) > 0:

      print("Ingredients extraient : ", ingredients)

      recipeEmbedding = generateEmbeddings(" ".join(str(ingredient) for ingredient in ingredients))

      results = collection.query(
          query_embeddings= [recipeEmbedding],
          n_results= 5,
      )

      if len(results) == 0:
        return "Auun document trouvé"
      else:

        top_foodItems = [
            {
                'ids': [food_id for food_id in results['ids'][0]],
                'distances': [distance for distance in results['distances'][0]],
                'food_names': [food_name['food_name'] for food_name in results['metadatas'][0]]
            }
            for index, id in enumerate(results['ids'][0])
        ]
        print(top_foodItems)
        if top_foodItems:
          for index, item in enumerate(top_foodItems[0]["food_names"]):
            print(f"Top {index + 1} Recommended Food Name: {item}")
        else:
          print("Aucun résultat trouvé.")

    else:
      print("Aucun ingrédient trouvé")

  except Exception as e:

    print("Exception main ", e)

In [ ]:
main()

ok
Embeddings stocker dans chromaDB 
Ingredients extraient :  ['unsalted butter', 'large eggs', 'vanilla extract', 'cocoa powder', 'dark chocolate', 'granulated sugar', 'salt', 'almond flour', 'whipped cream', 'fresh berries', 'heavy cream']
[{'ids': ['chocolate_2_129', '2_1', 'chocolate_1_128', '16_15', '5_4'], 'distances': [0.47074493765830994, 0.4835055470466614, 0.554291844367981, 0.5568122863769531, 0.5622150897979736], 'food_names': ['Chocolate Lava Cake', 'Chocolate Cake', 'Classic Chocolate Cake', 'Profiteroles', 'Cheesecake']}, {'ids': ['chocolate_2_129', '2_1', 'chocolate_1_128', '16_15', '5_4'], 'distances': [0.47074493765830994, 0.4835055470466614, 0.554291844367981, 0.5568122863769531, 0.5622150897979736], 'food_names': ['Chocolate Lava Cake', 'Chocolate Cake', 'Classic Chocolate Cake', 'Profiteroles', 'Cheesecake']}, {'ids': ['chocolate_2_129', '2_1', 'chocolate_1_128', '16_15', '5_4'], 'distances': [0.47074493765830994, 0.4835055470466614, 0.554291844367981, 0.5568122863

In [ ]:
# supprimer la collection
chroma_client.delete_collection(collectionName)